# Notícias G1 - 6. Carga no Elastic Search

Será feita a carga dos dados extraídos a partir do portal do G1 em uma base de dados local com Elastic Search que é um banco no SQL e não relacional.

## Imports:

Importes de bibliotecas usadas com frequência no código.

In [1]:
import pandas as pd
import numpy as np

## Recuperando Dados Gravados no Arquivo:

Recuperando dados gravados após todos os tratamentos e higienizações necessárias.

In [2]:
noticias = pd.read_csv('noticias_g1_higienizadas.csv')
noticias

,titulo,url,texto,categoria,dado_tratado
0,Entenda como uma ampla coalizão tirou Netanyah...,https://g1.globo.com/mundo/noticia/2021/06/13/...,O Parlamento de Israel confirmou neste doming...,FALTANDO,"['entend', 'ampl', 'coaliz', 'tir', 'netanyahu..."
1,"Milionário de direita: quem é Naftali Bennett,...",https://g1.globo.com/mundo/noticia/2021/06/13/...,"Milionário do setor de tecnologia, Naftali Be...",FALTANDO,"['milionri', 'direit', 'naftal', 'bennett', 'n..."
2,Governo de SP antecipa datas de vacinação; vej...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,O governo de São Paulo antecipou em 30 dias o...,FALTANDO,"['govern', 'sp', 'antecip', 'dat', 'vacina', '..."
3,PMs que mataram homens dentro de carro em SP s...,https://g1.globo.com/sp/sao-paulo/noticia/2021...,A Polícia Militar instaurou um inquérito poli...,FALTANDO,"['pm', 'mat', 'hom', 'dentr', 'carr', 'sp', 's..."
4,"Após Venezuela e Bolívia, Colômbia confirma 2 ...",https://ge.globo.com/futebol/copa-america/noti...,A Federação Colombiana de Futebol informou ne...,esportes,"['ap', 'venezuel', 'bolv', 'colmb', 'confirm',..."
...,...,...,...,...,...
1907,Segundo suspeito de agredir cabo da PM em bloc...,https://g1.globo.com/rj/regiao-dos-lagos/notic...,O segundo suspeito de espancar um policial mi...,FALTANDO,"['segund', 'suspeit', 'agred', 'cab', 'pm', 'b..."
1908,O perigo de definirmos nossa identidade pelo t...,https://g1.globo.com/economia/concursos-e-empr...,O sobrenome mais popular na Alemanha e na Suí...,economia,"['perig', 'defin', 'ident', 'trabalh', 'sobren..."
1909,"Jogador Arboleda, do São Paulo, diz ter 'noção...",https://g1.globo.com/sp/sao-paulo/noticia/2021...,"O zagueiro equatoriano Arboleda, do São Paulo...",FALTANDO,"['jog', 'arboled', 'so', 'paul', 'diz', 'ter',..."
1910,Gilberto Gil lança álbum ao vivo em que faz fe...,https://g1.globo.com/pop-arte/musica/blog/maur...,♪ Sem alarde e somente com um ou outro aviso ...,pop-arte,"['gilbert', 'gil', 'lan', 'lbum', 'viv', 'faz'..."


## Conectando com o Elastic Search:

Estabelecendo a conexão para realizar as operações no banco de dados.

In [4]:
!pip install elasticsearch
import elasticsearch
client = elasticsearch.Elasticsearch('localhost:9200')
client.info()

{'name': 'elastic_node01',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'euW7vXsqSruqSHXTKey9IQ',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## Criando Índice no ELS:

Serão criados os índices no Elastic Search para que os dados sejam gravados. Isso poderia ser feito por inferência da base com relação aos tipos de dados, mas obtemos melhor desempenho fazendo os índices da forma mais adequada.

In [5]:
mapping = {
    "settings": {
        "number_of_shards": 3
    },
    "mappings": {
        "properties": {
            "titulo": { "type": "text", "fielddata": "true" },
            "url": { "type": "text" },
            "texto": { "type": "text", "fielddata": "true" },
            "categoria": { "type": "keyword" },
            "texto_tratado": { "type": "text" },
        }
    }
}

index_name = "noticias_g1"

client.indices.delete(index=index_name)

response = client.indices.create(
    index = index_name,
    body = mapping,
    ignore=400 # ignore already exists code
)

print ('Response:', response)

if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])

elif 'error' in response:
    print ("ERROR:", response['error']['root_cause'])
    print ("TYPE:", response['error']['type'])

Response: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'noticias_g1'}
INDEX MAPPING SUCCESS FOR INDEX: noticias_g1


## Carga de Dados no ELS:

Será feita a carga dos dados que estão no DataFrame na base de dados do Elastic Search.

In [6]:
from elasticsearch import helpers

def fields(document):
    return { key: document[key] for key in ['titulo','url','texto','categoria','texto_tratado'] }

def doc_generator(df):
    id = 0
    for index, document in df.iterrows():
        id = id + 1
        yield {
            "_index": f"{index_name}",
            "_id" : f"{id}",
            "_source": fields(document)
        }
    return

helpers.bulk(client, doc_generator(noticias))

(1953, [])

## Teste de Consulta:

Vamos realizar uma consulta básica na base de dados para certificar que a carga ocorreu corretamente.

In [8]:
teste = client.get(index=index_name, doc_type='_doc', id=1)
teste['_source']['url']

'https://g1.globo.com/politica/noticia/2021/03/09/a-pedido-de-nunes-marques-conclusao-do-julgamento-da-suspeicao-de-moro-e-adiado.ghtml'